In [4]:
import torch

In [9]:
import torchvision

In [8]:
from torch import nn

In [10]:
from torchvision import transforms


In [11]:
from torch.utils.data import DataLoader


In [13]:
transform=transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5],std=[0.5])
])

In [14]:
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 53364315.44it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 1781852.73it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 12690686.98it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 3552877.43it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [16]:
print(len(train_dataset))
print(len(test_dataset))

60000
10000


In [42]:
class Networl(nn.Module):
    def __init__(self):
        super(Networl,self).__init__()
        self.layer1=nn.Linear(784,256)
        self.layer2=nn.Linear(256,10)
    def forward(self,x):
        #x=x.view(-1,28,28)
        x=x.view(-1,784)
        x=self.layer1(x)
        x=torch.relu(x)
        x=self.layer2(x)
        return x

In [45]:
train_loader=DataLoader(train_dataset,batch_size=64,shuffle=True)
print(len(train_loader))
test_loader = DataLoader(dataset=test_dataset, batch_size=1000, shuffle=False)

938


In [44]:
for batch_idx,(data,label) in enumerate(train_loader):
    if batch_idx == 3:
        break
    print(batch_idx)
    print(data.shape)
    print(label.shape)
    print(label)

0
torch.Size([64, 1, 28, 28])
torch.Size([64])
tensor([7, 7, 1, 8, 1, 8, 3, 0, 3, 2, 3, 0, 9, 6, 7, 8, 7, 0, 4, 3, 2, 6, 9, 3,
        5, 9, 3, 0, 7, 6, 4, 3, 8, 9, 7, 4, 0, 9, 9, 7, 9, 8, 9, 9, 2, 5, 0, 5,
        2, 4, 2, 6, 9, 4, 6, 2, 8, 6, 6, 3, 6, 1, 5, 8])
1
torch.Size([64, 1, 28, 28])
torch.Size([64])
tensor([4, 3, 6, 8, 7, 2, 4, 9, 9, 3, 6, 0, 8, 1, 6, 3, 9, 0, 5, 7, 6, 8, 1, 6,
        2, 0, 4, 3, 2, 0, 4, 0, 2, 2, 0, 7, 7, 4, 3, 2, 3, 6, 7, 4, 6, 9, 3, 2,
        2, 2, 2, 5, 1, 2, 8, 2, 6, 9, 9, 1, 7, 6, 3, 2])
2
torch.Size([64, 1, 28, 28])
torch.Size([64])
tensor([8, 4, 6, 1, 8, 5, 5, 0, 9, 8, 0, 8, 2, 6, 6, 0, 1, 1, 1, 1, 5, 6, 2, 1,
        4, 7, 4, 9, 7, 2, 6, 8, 7, 0, 4, 9, 0, 2, 3, 3, 4, 4, 3, 9, 0, 6, 3, 6,
        6, 0, 4, 3, 7, 8, 5, 7, 8, 6, 1, 3, 2, 8, 2, 8])


In [46]:
model=Networl()

In [47]:
optimizer= torch.optim.Adam(model.parameters(),lr=0.001)

In [48]:
criterion=nn.CrossEntropyLoss()

In [57]:
def train(epoch):
  model.train()
  for batch_idx,(data,label) in enumerate(train_loader):
      optimizer.zero_grad()
      output=model(data)
      loss=criterion(output,label)
      loss.backward()
      optimizer.step()
      if batch_idx % 100 == 0:
        print(f'Train Epoch: {epoch}'
        f'{batch_idx}/{len(train_loader)}'
        f'Loss: {loss.item():.6f}')
  torch.save(model.state_dict(), 'model.pth')

In [58]:
train(10)

Train Epoch: 100/938Loss: 0.142876
Train Epoch: 10100/938Loss: 0.076827
Train Epoch: 10200/938Loss: 0.119318
Train Epoch: 10300/938Loss: 0.092391
Train Epoch: 10400/938Loss: 0.167190
Train Epoch: 10500/938Loss: 0.226701
Train Epoch: 10600/938Loss: 0.392473
Train Epoch: 10700/938Loss: 0.099284
Train Epoch: 10800/938Loss: 0.080751
Train Epoch: 10900/938Loss: 0.071037


In [64]:
model.load_state_dict(torch.load('model.pth'))
model.eval()

Networl(
  (layer1): Linear(in_features=784, out_features=256, bias=True)
  (layer2): Linear(in_features=256, out_features=10, bias=True)
)

In [69]:
def test():
  right=0
  for i,(x,y) in enumerate(test_dataset):
    output=model(x)
    predict=output.argmax(1).item()
    if predict==y:
      right+=1
    else:
      print(f'predict:{predict},label:{y}')
  print(right/len(test_dataset))

In [70]:
test()

predict:6,label:5
predict:4,label:9
predict:4,label:2
predict:1,label:3
predict:2,label:4
predict:0,label:6
predict:7,label:2
predict:9,label:7
predict:7,label:3
predict:0,label:6
predict:5,label:3
predict:6,label:5
predict:0,label:8
predict:2,label:8
predict:3,label:8
predict:9,label:7
predict:8,label:2
predict:6,label:2
predict:8,label:2
predict:1,label:7
predict:4,label:8
predict:8,label:5
predict:1,label:3
predict:7,label:8
predict:4,label:9
predict:7,label:9
predict:9,label:8
predict:9,label:7
predict:4,label:5
predict:0,label:6
predict:3,label:5
predict:5,label:6
predict:8,label:5
predict:1,label:7
predict:8,label:6
predict:6,label:4
predict:7,label:3
predict:9,label:7
predict:8,label:6
predict:4,label:9
predict:9,label:7
predict:9,label:7
predict:2,label:7
predict:4,label:9
predict:6,label:5
predict:3,label:9
predict:3,label:8
predict:1,label:2
predict:1,label:7
predict:2,label:7
predict:9,label:5
predict:7,label:5
predict:3,label:8
predict:6,label:8
predict:1,label:7
predict:9,

In [ ]:
#accuracy is 0.9653,a simple code file